In [5]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Reshape, BatchNormalization, Dense
from sklearn.model_selection import train_test_split
from scipy import stats
from ncps.tf import LTC
from tensorflow.keras.callbacks import ModelCheckpoint
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Function to scale input data
def scaled_in(matrix_spec):
    "Global scaling applied to noisy voice spectrograms (scale between -1 and 1)"
    matrix_spec = (matrix_spec + 46) / 50
    return matrix_spec

# Function to scale output data
def scaled_ou(matrix_spec):
    "Global scaling applied to noise models spectrograms (scale between -1 and 1)"
    matrix_spec = (matrix_spec - 6) / 82
    return matrix_spec

# Load data
path_save_spectrogram = '/content/drive/MyDrive/npy/'  # Specify the path to the spectrogram directory
X_in = np.load(os.path.join(path_save_spectrogram, 'noisy_voice_amp_db.npy'))
X_ou = np.load(os.path.join(path_save_spectrogram, 'voice_amp_db.npy'))

# Model of noise to predict
X_ou = X_in - X_ou

# Check distribution
print(stats.describe(X_in.reshape(-1, 1)))
print(stats.describe(X_ou.reshape(-1, 1)))

# Scale input and output data
X_in = scaled_in(X_in)
X_ou = scaled_ou(X_ou)

# Check shape of spectrograms
print(X_in.shape)
print(X_ou.shape)
# Check new distribution
print(stats.describe(X_in.reshape(-1, 1)))
print(stats.describe(X_ou.reshape(-1, 1)))

# Reshape for training
X_in = X_in[:, :, :]
X_in = X_in.reshape(X_in.shape[0], X_in.shape[1], X_in.shape[2], 1)
X_ou = X_ou[:, :, :]
X_ou = X_ou.reshape(X_ou.shape[0], X_ou.shape[1], X_ou.shape[2], 1)

# Split data into train and validation sets
x_train, x_val, y_train, y_val = train_test_split(X_in, X_ou, test_size=0.10, random_state=42)

# Load the saved UNet model
unet_model = load_model(os.path.join(path_save_spectrogram, 'weights/model_ResNet.keras'))

# Define RNN model
ncp = LTC(32, 16)
rnn_model = tf.keras.Sequential()
rnn_model.add(Reshape((-1, 1)))  # Reshape to add a time dimension
rnn_model.add(ncp)
rnn_model.add(BatchNormalization())
rnn_model.add(Dense(11, activation='softmax'))

# Define the combined model
combined_output = rnn_model(unet_model.output)
combined_model = Model(inputs=unet_model.input, outputs=combined_output)

# Compile the combined model
combined_model.compile(optimizer='Adam',
                       loss=tf.keras.losses.MeanSquaredError(),
                       metrics=['mae'])

# Define filepath for saving the best model
model_checkpoint_path = 'best_model_combined.keras'

# Define the ModelCheckpoint callback
checkpoint = ModelCheckpoint(model_checkpoint_path,
                             verbose=1,
                             monitor='val_loss',
                             save_best_only=True,
                             mode='auto')

# Train the combined model
history = combined_model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=20, batch_size=10, verbose=1, callbacks=[checkpoint])

# Save model architecture to JSON file
model_json = combined_model.to_json()
with open('model_combined.json', 'w') as json_file:
    json_file.write(model_json)

# Save model weights
combined_model.save_weights('model_combined_weights.keras')

# Plot training and validation loss (log scale)
import matplotlib.pyplot as plt

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, label='Training loss')
plt.plot(epochs, val_loss, label='Validation loss')
plt.yscale('log')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Print epoch information and other variables
print("Epochs:", epochs)
print("Training Loss:", loss)
print("Validation Loss:", val_loss)


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24_input (InputLaye  [(None, 128, 128, 1)]     0         
 r)                                                              
                                                                 
 conv2d_24 (Conv2D)          (None, 128, 128, 32)      320       
                                                                 
 conv2d_25 (Conv2D)          (None, 128, 128, 32)      9248      
                                                                 
 max_pooling2d_12 (MaxPooli  (None, 64, 64, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_26 (Conv2D)          (None, 64, 64, 64)        18496     
                                                                 
 conv2d_27 (Conv2D)          (None, 64, 64, 64)        3692

In [ ]:
def training_cnn(path_save_spectrogram, weights_path, epochs, batch_size):
    # Load noisy voice & clean voice spectrograms created by data_creation mode
    X_in = np.load(path_save_spectrogram +'noisy_voice_amp_db'+".npy")
    X_ou = np.load(path_save_spectrogram +'voice_amp_db'+".npy")

    # Model of noise to predict
    X_ou = X_in - X_ou

    # Check distribution
    print(stats.describe(X_in.reshape(-1,1)))
    print(stats.describe(X_ou.reshape(-1,1)))

    # Rescale between -1 and 1
    X_in = scaled_in(X_in)
    X_ou = scaled_ou(X_ou)

    # Check shape of spectrograms
    print(X_in.shape)
    print(X_ou.shape)
    # Check new distribution
    print(stats.describe(X_in.reshape(-1,1)))
    print(stats.describe(X_ou.reshape(-1,1)))

    # Reshape for training
    X_in = X_in[:, :, :]
    X_in = X_in.reshape(X_in.shape[0], X_in.shape[1], X_in.shape[2], 1)
    X_ou = X_ou[:, :, :]
    X_ou = X_ou.reshape(X_ou.shape[0], X_ou.shape[1], X_ou.shape[2], 1)

    X_train, X_test, y_train, y_test = train_test_split(X_in, X_ou, test_size=0.10, random_state=42)
    
    print("X_train shape:", X_train.shape)
    print("X_test shape:", X_test.shape)
    print("y_train shape:", y_train.shape)
    print("y_test shape:", y_test.shape)

    # Create CNN model
    model = cnn_model(X_train.shape[1:])

    # Save best models to disk during training
    checkpoint = ModelCheckpoint(weights_path + '/model_cnn_best.h5', verbose=1, monitor='val_loss', save_best_only=True, mode='auto')

    model.summary()

    # Training
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, shuffle=True, callbacks=[checkpoint], verbose=1, validation_data=(X_test, y_test))

    # Saving Model
    model.save_weights(weights_path + '/model_cnn_final.h5')

    # Plot training and validation loss (log scale)
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs = range(1, len(loss) + 1)

    plt.plot(epochs, loss, label='Training loss')
    plt.plot(epochs, val_loss, label='Validation loss')
    plt.yscale('log')
    plt.title('Training and validation loss')
    plt.legend()
    plt.show()

# Example usage:
# Assuming you have the necessary data files and directories set up

# Set the paths and parameters
path_save_spectrogram = 'spectogram/'
weights_path = 'weights/'
epochs = 20
batch_size = 10

# Call the training function
training_cnn(path_save_spectrogram, weights_path, epochs, batch_size)